In [5]:
from src.data_examples.ex2_data_loader_1m import ExampleDataLoader

In [6]:
data_loader = ExampleDataLoader()
data_loader.download().load()

downloading file to cache\datasets\movielens\ml-1m.zip
file size: 5917549
progress: [===                 ] 17.72%
progress: [=======             ] 35.44%
progress: [==========          ] 53.16%
progress: [==============      ] 70.88%
progress: [=================   ] 88.60%
progress: [====================] 100.00%
extracting zip file content:
extracting - size: 0 - filename: ml-1m/
extracting - size: 171308 - filename: ml-1m/movies.dat
extracting - size: 24594131 - filename: ml-1m/ratings.dat
extracting - size: 5577 - filename: ml-1m/README
extracting - size: 134368 - filename: ml-1m/users.dat


c:\Users\62896\Documents\GitHub\FederatedLearning\src\data_examples\ex2_data_loader_1m.py:107: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(
c:\Users\62896\Documents\GitHub\FederatedLearning\src\data_examples\ex2_data_loader_1m.py:107: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  df = pd.read_csv(
c:\Users\62896\Documents\GitHub\FederatedLearning\src\data_examples\ex2_data_loader_1m.py:107: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(
c:\Users\62896\Documents\GitHub\FederatedLearning\src\data_example

In [7]:
data_loader.df_users

,user_id,gender,age,occupation,zip_code
0,0,F,1,10,48067
1,1,M,56,16,70072
2,2,M,25,15,55117
3,3,M,45,7,02460
4,4,M,25,20,55455
...,...,...,...,...,...
6035,6035,F,25,15,32603
6036,6036,F,45,1,76006
6037,6037,F,56,1,14706
6038,6038,F,45,0,01060
